In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType

from ibm_botocore.client import Config
import ibm_boto3
import os
from pyspark.sql.functions import unix_timestamp, to_date
import pyspark.sql.functions as F


import numpy as np
import pandas as pd
import json
import pyspark.sql
from pyspark.sql.functions import col, skewness, kurtosis
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import when
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType
from datetime import date, timedelta, datetime
import time

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210210214756-0000
KERNEL_ID = cf26e6a5-ee22-4eef-b657-4a2c92476544


Hidden cell with credentials for accessing from Cloud Object Storage

In [2]:
# The code was removed by Watson Studio for sharing.

In [4]:
#cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')
#ratings_df = spark.read.parquet(cos.url('ratingsdataframe.parquet', <BUCKET NAME>))

ratings_df.createOrReplaceTempView("ratings")
ratings_df.take(5)

[Row(userid='1', movieid='1193', rating='5', timestamp='978300760'),
 Row(userid='1', movieid='661', rating='3', timestamp='978302109'),
 Row(userid='1', movieid='914', rating='3', timestamp='978301968'),
 Row(userid='1', movieid='3408', rating='4', timestamp='978300275'),
 Row(userid='1', movieid='2355', rating='5', timestamp='978824291')]

In [5]:
#movies_df = spark.read.parquet(cos.url('moviesdataframe.parquet', <BUCKET NAME>))
movies_df.createOrReplaceTempView("movies")
movies_df.take(5)

[Row(movieid='1', title='Toy Story (1995)', year='1995', genre="Animation|Children's|Comedy"),
 Row(movieid='2', title='Jumanji (1995)', year='1995', genre="Adventure|Children's|Fantasy"),
 Row(movieid='3', title='Grumpier Old Men (1995)', year='1995', genre='Comedy|Romance'),
 Row(movieid='4', title='Waiting to Exhale (1995)', year='1995', genre='Comedy|Drama'),
 Row(movieid='5', title='Father of the Bride Part II (1995)', year='1995', genre='Comedy')]

In [12]:
result = spark.sql("""
            Select movies.title as Movie,COUNT(ratings.userid) as Total_number_of_Users
            FROM movies INNER JOIN ratings ON movies.movieid=ratings.movieid
            GROUP BY movies.title
            ORDER BY movies.title ASC
            """).repartition(1)


result.show()


+--------------------+---------------------+
|               Movie|Total_number_of_Users|
+--------------------+---------------------+
|$1,000,000 Duck (...|                   37|
|'Night Mother (1986)|                   70|
|'Til There Was Yo...|                   52|
|  'burbs, The (1989)|                  303|
|...And Justice fo...|                  199|
|        1-900 (1994)|                    2|
|10 Things I Hate ...|                  700|
|101 Dalmatians (1...|                  565|
|101 Dalmatians (1...|                  364|
| 12 Angry Men (1957)|                  616|
|13th Warrior, The...|                  750|
|          187 (1997)|                   55|
|2 Days in the Val...|                  286|
|     20 Dates (1998)|                  139|
|20,000 Leagues Un...|                  575|
|200 Cigarettes (1...|                  181|
|2001: A Space Ody...|                 1716|
|         2010 (1984)|                  470|
|24 7: Twenty Four...|                    5|
|24-hour W

In [13]:
## To save it locally
from IPython.display import HTML 
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "result.csv"):  
    csv = df.to_csv(sep=',',header=True, index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)


# sparkDF to Pandas
result_df = result.toPandas()
create_download_link(result_df,filename='users_per_movie.csv')